<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Preprocess Data
</div>

In [1]:
import pandas as pd

### Read raw data that we have collected

In [2]:
annual_pop_df = pd.read_csv("../data/raw/OA.csv")
agri_employments_df = pd.read_csv("../data/raw/OEA.csv")
rural_employments_df = pd.read_csv("../data/raw/OER.csv")
values_df = pd.read_csv("../data/raw/QCL.csv")
products_df = pd.read_csv("../data/raw/QV.csv")

display(annual_pop_df.head(3))
display(agri_employments_df.head(3))
display(rural_employments_df.head(3))
display(values_df.head(3))
display(products_df.head(3))

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,OA,Annual population,237,Viet Nam,511,Total Population - Both sexes,3010,Population - Est. & Proj.,1950,1950,1000 No,25109.200,X,Figure from international organizations,NaN
1,OA,Annual population,237,Viet Nam,512,Total Population - Male,3010,Population - Est. & Proj.,1950,1950,1000 No,12408.031,X,Figure from international organizations,NaN
2,OA,Annual population,237,Viet Nam,513,Total Population - Female,3010,Population - Est. & Proj.,1950,1950,1000 No,12701.169,X,Figure from international organizations,NaN


,Domain Code,Domain,Area Code,Area,Indicator Code,Indicator,Sex Code,Sex,Year Code,Year,Element Code,Element,Source Code,Source,Unit,Value,Flag,Flag Description,Note
0,OEA,Employment Indicators: Agriculture,237,Viet Nam,21144,"Employment in agriculture, forestry and fishin...",1,Total,1991,1991,6199,Value,3043,ILO - ILO Modelled Estimates,1000 No,24059.27,X,Figure from international organizations,NaN
1,OEA,Employment Indicators: Agriculture,237,Viet Nam,21144,"Employment in agriculture, forestry and fishin...",1,Total,1992,1992,6199,Value,3043,ILO - ILO Modelled Estimates,1000 No,24384.94,X,Figure from international organizations,NaN
2,OEA,Employment Indicators: Agriculture,237,Viet Nam,21144,"Employment in agriculture, forestry and fishin...",1,Total,1993,1993,6199,Value,3043,ILO - ILO Modelled Estimates,1000 No,24641.43,X,Figure from international organizations,NaN


,Domain Code,Domain,Area Code,Area,Indicator Code,Indicator,Sex Code,Sex,Year Code,Year,Element Code,Element,Source Code,Source,Unit,Value,Flag,Flag Description,Note
0,OER,Employment Indicators: Rural,237,Viet Nam,21087,"Employment by age, total (15+), rural areas",1,Total,2007,2007,6199,Value,3023,Labour force survey,1000 No,34098.44,X,Figure from international organizations,Break in series: Methodology revised Repositor...
1,OER,Employment Indicators: Rural,237,Viet Nam,21087,"Employment by age, total (15+), rural areas",1,Total,2009,2009,6199,Value,3023,Labour force survey,1000 No,34615.17,X,Figure from international organizations,Repository: ILO-STATISTICS - Micro data proces...
2,OER,Employment Indicators: Rural,237,Viet Nam,21087,"Employment by age, total (15+), rural areas",1,Total,2010,2010,6199,Value,3023,Labour force survey,1000 No,35839.50,X,Figure from international organizations,Repository: ILO-STATISTICS - Micro data proces...


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,ha,1000.0,E,Estimated value,NaN
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,100 g/ha,7000.0,E,Estimated value,NaN
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,t,700.0,E,Estimated value,NaN


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QV,Value of Agricultural Production,237,Viet Nam,152,Gross Production Value (constant 2014-2016 tho...,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,1000 Int. $,1521,E,Estimated value
1,QV,Value of Agricultural Production,237,Viet Nam,152,Gross Production Value (constant 2014-2016 tho...,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1962,1000 Int. $,1521,E,Estimated value
2,QV,Value of Agricultural Production,237,Viet Nam,152,Gross Production Value (constant 2014-2016 tho...,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,1963,1000 Int. $,1521,E,Estimated value


### Clear duplicates columns:
-   There is `Year Code` and `Year` columns, they have the same values, so we drop the `Year Code`.

In [3]:
annual_pop_df = annual_pop_df.drop(columns=['Year Code'])
agri_employments_df = agri_employments_df.drop(columns=['Year Code'])
rural_employments_df = rural_employments_df.drop(columns=['Year Code'])
values_df = values_df.drop(columns=['Year Code'])
products_df = products_df.drop(columns=['Year Code'])

### Check for duplicate rows

In [4]:
for df in [annual_pop_df, agri_employments_df, rural_employments_df, values_df, products_df]:
    index = agri_employments_df.index
    detectDupSeries = index.duplicated(keep='first')
    num_duplicated_rows = detectDupSeries.sum()
    if num_duplicated_rows == 0:
        print(f"{df['Domain'].values[0]} have no duplicated line.!")
    else:
        if num_duplicated_rows > 1:
            ext = "lines"
        else:
            ext = "line"
        print(f"{df['Domain'].values[0]} {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

Annual population have no duplicated line.!
Employment Indicators: Agriculture have no duplicated line.!
Employment Indicators: Rural have no duplicated line.!
Crops and livestock products have no duplicated line.!
Value of Agricultural Production have no duplicated line.!


### Determine the features that have a large number of missing values

In [5]:
for df in [annual_pop_df, agri_employments_df, rural_employments_df, values_df, products_df]:
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                    'percent_missing': percent_missing})
    
    miss = (missing_value_df['column_name'][missing_value_df['percent_missing'].values >= 75]).values
    print(f"{df['Domain'].values[0]}, features with large missing value: {miss} (>= 75%)")

Annual population, features with large missing value: ['Note'] (>= 75%)
Employment Indicators: Agriculture, features with large missing value: [] (>= 75%)
Employment Indicators: Rural, features with large missing value: [] (>= 75%)
Crops and livestock products, features with large missing value: ['Note'] (>= 75%)
Value of Agricultural Production, features with large missing value: [] (>= 75%)


-   `annual_pop_df` and `products_df` have `Note` as the features that have a large number of missing values, but for now, we will keep it

### Check data type of each columns

In [6]:
for df in [annual_pop_df, agri_employments_df, rural_employments_df, values_df, products_df]:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       655 non-null    object 
 1   Domain            655 non-null    object 
 2   Area Code         655 non-null    int64  
 3   Area              655 non-null    object 
 4   Element Code      655 non-null    int64  
 5   Element           655 non-null    object 
 6   Item Code         655 non-null    int64  
 7   Item              655 non-null    object 
 8   Year              655 non-null    int64  
 9   Unit              655 non-null    object 
 10  Value             655 non-null    float64
 11  Flag              655 non-null    object 
 12  Flag Description  655 non-null    object 
 13  Note              0 non-null      float64
dtypes: float64(2), int64(4), object(8)
memory usage: 71.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data c

-   For all of the above dataframe: `Area Code`, `Element Code`, `Item Code`, `Year` , `Indicator Code` have numeric type. However, their magnitude does not have a significance. And, `Year` actually represents a period instead of a number, `Area Code`, `Element Code`, `Item Code`, `Indicator Code` are just a category. Thus, they can be convert to categorical type.

In [7]:
for col in annual_pop_df.drop(columns= 'Value').columns:
    annual_pop_df[col] = annual_pop_df[col].astype(str)

for col in agri_employments_df.drop(columns= 'Value').columns:
    agri_employments_df[col] = agri_employments_df[col].astype(str)

for col in rural_employments_df.drop(columns= 'Value').columns:
    rural_employments_df[col] = rural_employments_df[col].astype(str)

for col in values_df.drop(columns= 'Value').columns:
    values_df[col] = values_df[col].astype(str)

for col in products_df.drop(columns= 'Value').columns:
    products_df[col] = products_df[col].astype(str)

-   Let check one last time

In [8]:
for df in [annual_pop_df, agri_employments_df, rural_employments_df, values_df, products_df]:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       655 non-null    object 
 1   Domain            655 non-null    object 
 2   Area Code         655 non-null    object 
 3   Area              655 non-null    object 
 4   Element Code      655 non-null    object 
 5   Element           655 non-null    object 
 6   Item Code         655 non-null    object 
 7   Item              655 non-null    object 
 8   Year              655 non-null    object 
 9   Unit              655 non-null    object 
 10  Value             655 non-null    float64
 11  Flag              655 non-null    object 
 12  Flag Description  655 non-null    object 
 13  Note              655 non-null    object 
dtypes: float64(1), object(13)
memory usage: 71.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (t

### Save processed data to a new csv file

In [9]:
annual_pop_df.to_csv("../data/processed/preprocess_OA.csv", index= False)
agri_employments_df.to_csv("../data/processed/preprocess_OEA.csv", index= False)
rural_employments_df.to_csv("../data/processed/preprocess_OER.csv", index= False)
values_df.to_csv("../data/processed/preprocess_QV.csv", index= False)
products_df.to_csv("../data/processed/preprocess_QCL.csv", index= False)